In [1]:
# https://www.kaggle.com/sudalairajkumar/covid19-in-italy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import geopandas as gpd
from PIL import Image
import imageio

df = pd.read_csv("data/covid19_italy_region.csv")
df.tail()

,SNo,Date,Country,RegionCode,RegionName,Latitude,Longitude,HospitalizedPatients,IntensiveCarePatients,TotalHospitalizedPatients,HomeConfinement,CurrentPositiveCases,NewPositiveCases,Recovered,Deaths,TotalPositiveCases,TestsPerformed
982,982,2020-04-10T17:00:00,ITA,9,Toscana,43.769231,11.255889,1011,256,1267,4555,5822,175,451,454,6727,72001
983,983,2020-04-10T17:00:00,ITA,4,P.A. Trento,46.068935,11.121231,335,70,405,1589,1994,108,547,275,2816,14981
984,984,2020-04-10T17:00:00,ITA,10,Umbria,43.106758,12.388247,146,39,185,567,752,4,498,52,1302,16325
985,985,2020-04-10T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,131,16,147,455,602,11,170,107,879,3328
986,986,2020-04-10T17:00:00,ITA,5,Veneto,45.434905,12.338452,1521,257,1778,8869,10647,488,1981,793,13421,180700


In [2]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world = world[~world.name.isin(["Antarctica", "Fr. S. Antarctic Lands"])]

In [3]:
def plot_map(df, i):
    fig, ax = plt.subplots(1, figsize=(14,11))

    # plot map
    world.plot(ax=ax, color='black')
    
    # plot infected locations
    plt.scatter(x=df['Longitude'], y=df['Latitude'], color='orangered', 
            s=df['TotalPositiveCases'] /50)
    
    plt.scatter(x=df['Longitude'], y=df['Latitude'], color='lightblue', 
            s=(df['Recovered']+df['Deaths']) /50)
    
    plt.scatter(x=df['Longitude'], y=df['Latitude'], color='darkgrey', 
            s=df['Deaths'] /50)
    
    ax.legend(['Confirmed', 'Recovered', 'Deaths'], fontsize = 24, markerscale = 0.5)
    
    plt.xlim([5,20])
    plt.ylim([36,48])
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    # hide map axis
    ax.axis('off')
    plt.text(6, 47, 'Confirmed Cases at '+str(i)[:10], color='white', fontsize=24)
    plt.text(6, 46.2, '{:,}'.format(df['TotalPositiveCases'].sum()), color='white', fontsize=24)
    plt.title("Italy's Covid-19 cases by region")
    plt.savefig('pics/ita/'+ str(i)[:10]+'_ita.png', facecolor='white', edgecolor='none')
    

def plot_tests(x):
    fig, ax = plt.subplots(1, figsize=(14,8))

    tests = df.groupby('Date').sum()['TestsPerformed']
    tests.values
    tests_daily = [tests.values[0]]
    for i, val in enumerate(tests.values[:-1]):
        tests_daily.append(tests.values[i+1]-val )

    pos = df.groupby('Date').sum()['NewPositiveCases']    

    plt.bar( np.arange(len(tests_daily))[:x] , tests_daily[:x] )
    plt.bar( np.arange(len(tests_daily))[:x] , pos.values[:x], color='orangered' )

    # remove spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)

    # grid
    ax.grid(color='grey', linestyle='dashed', linewidth=1, axis = 'x')
    ax.set_axisbelow(True)

    #labels
    plt.ylabel('Daily Amount', fontsize = 24)
    plt.legend(['Tests','Positives'], fontsize = 24)
    plt.xlim([-0.5,len(tests_daily)])
    plt.xticks(fontsize = 19)
    plt.yticks(fontsize = 19)
    
    plt.subplots_adjust(left=0.1, right=0.95, top=1, bottom=0.08)
    plt.savefig('pics/ita/'+ str(x)+'_tests_ita.png', facecolor='white', edgecolor='none')

In [4]:

def build_gif(filenames, title, duration=1):
    images = []
    for filename in filenames:
        images.append(imageio.imread(filename))

    imageio.mimwrite((title + '.gif'), images, duration=duration)

In [5]:
def build_collage(file1, file2, i):
    im1 = Image.open(file1)
    im2 = Image.open(file2)

    dst = Image.new('RGB', (im1.width, im1.height + im2.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))

    dst.save('pics/ita/'+ str(i) +'.png')
    
build_collage('pics/ita/2020-04-03_ita.png', 'pics/ita/38_tests_ita.png', 1)

In [9]:
date = df.Date.unique()

#plot_map(df[df['Date'] == date[-1]], date[-1] )
#plot_tests(len(date))
files = []
for i, day in enumerate(date):
    plot_map(df[df['Date'] == day], day )
    plot_tests(i+1)
    
    plt.close('all')
    
    file1 = 'pics/ita/'+ str(day)[:10]+'_ita.png'
    file2 = 'pics/ita/'+ str(i+1)+'_tests_ita.png'
    
    build_collage(file1, file2, i)
    
    files.append('pics/ita/'+ str(i) +'.png')

In [10]:
for i in np.arange(10):
    files.append('pics/ita/46.png')

build_gif(files, 'Italy_prog', 0.6)